In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
#import duckdb as dd
#from tqdm import tqdm
import matplotlib.pyplot as plt
#import cv2
#from pydicom import dcmread
import warnings
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import pickle
#import gc
import ctypes
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1727879932.197900      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1002 14:38:52.208950767      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1002 14:38:52.208966692      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1002 14:38:52.208969937      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1002 14:38:52.208972381      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1002 14:38:52.208974764      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1727879956.287795      13 service.cc:145] XLA service 0x5b1c46c3ded0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727879956.287856      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1727879956.287861      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1727879956.287864      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1727879956.287867      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1727879956.287869      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1727879956.287872      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1727879956.287875      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1727879956.287877      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
def read_and_parse_dicom_files_tensorflow_train(full_file_path):
    #tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    sqzd_image = tf.squeeze(image)
    #shaped_image = sqzd_image.set_shape(sqzd_image.get_shape())
    return sqzd_image

def read_parse_n_augment_dicom_files(full_file_path):
    #tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    
    brt_image = tf.image.adjust_brightness(image, delta=0.3)
    contrast_img = tf.image.adjust_contrast(brt_image, contrast_factor=2.0)
    contrast_img_resized = tf.image.resize(contrast_img, (128,128))

    return tf.squeeze(contrast_img_resized)

def preprocessing_aug_image(img_path):
    train_img = read_parse_n_augment_dicom_files(img_path)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_aug_dataset(image_path, labels):
    image = preprocessing_aug_image(image_path)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def preprocessing(img_path):
    train_img = read_and_parse_dicom_files_tensorflow_train(img_path)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train(image_path, labels):
    image = preprocessing(image_path)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

In [5]:
condition_for_training = 'right_neural_foraminal_narrowing'
vertebrae_position = 'l2_l3'

metadata_file_path = '/kaggle/input/right-neural-foraminal-narrowing-metadata/{0}_{1}_feature_metadata.csv'.format(condition_for_training, vertebrae_position)
metadata_df = pl.read_csv(metadata_file_path, low_memory=True)

metadata_df_severe = metadata_df.filter(pl.col('encoded_severity')==2)
metadata_df_moderate = metadata_df.filter(pl.col('encoded_severity')==0)

x_train, x_test_val = train_test_split(metadata_df, test_size=0.4, random_state=42)
x_test, x_valid = train_test_split(x_test_val, test_size=0.2, random_state=42)

print("Training data shape : {0}".format(x_train.shape))
print("Test data shape : {0}".format(x_test.shape))
print("Validation data shape : {0}".format(x_valid.shape))
print("metadata_df_severe shape : {0}".format(metadata_df_severe.shape))
print("metadata_df_moderate shape : {0}".format(metadata_df_moderate.shape))

Training data shape : (88000, 3)
Test data shape : (46934, 3)
Validation data shape : (11734, 3)
metadata_df_severe shape : (508, 3)
metadata_df_moderate shape : (13272, 3)


In [6]:
def generate_tf_datasets(p_train_df, p_test_df, p_valid_df, p_BATCH_SIZE_PER_REPLICA):

    BATCH_SIZE = p_BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
    
    train_image_filenames = pl.Series(p_train_df.select(pl.col('full_img_path'))).to_list()
    train_image_labels = pl.Series(p_train_df.select(pl.col('encoded_severity'))).to_list()

    test_image_filenames = pl.Series(p_test_df.select(pl.col('full_img_path'))).to_list()
    test_image_labels = pl.Series(p_test_df.select(pl.col('encoded_severity'))).to_list()

    valid_image_filenames = pl.Series(p_valid_df.select(pl.col('full_img_path'))).to_list()
    valid_image_labels = pl.Series(p_valid_df.select(pl.col('encoded_severity'))).to_list()
    
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_image_filenames, train_image_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((test_image_filenames, test_image_labels))
    valid_dataset = tf.data.Dataset.from_tensor_slices((valid_image_filenames, valid_image_labels))
    
    train_ds = train_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

    test_ds = test_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

    valid_ds = valid_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    valid_ds = valid_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    valid_ds = valid_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)
    
    return train_ds, test_ds, valid_ds

In [7]:
train_ds, test_ds, valid_ds = generate_tf_datasets(p_train_df=x_train, p_test_df=x_test, p_valid_df=x_valid
                                                   , p_BATCH_SIZE_PER_REPLICA = 16)

BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync

metadata_df_severe_filenames = pl.Series(metadata_df_severe.select(pl.col('full_img_path'))).to_list()
metadata_df_severe_labels = pl.Series(metadata_df_severe.select(pl.col('encoded_severity'))).to_list()

metadata_df_moderate_filenames = pl.Series(metadata_df_moderate.select(pl.col('full_img_path'))).to_list()
metadata_df_moderate_labels = pl.Series(metadata_df_moderate.select(pl.col('encoded_severity'))).to_list()

metadata_df_severe_dataset = tf.data.Dataset.from_tensor_slices((metadata_df_severe_filenames, metadata_df_severe_labels))
metadata_df_moderate_dataset = tf.data.Dataset.from_tensor_slices((metadata_df_moderate_filenames, metadata_df_moderate_labels))

metadata_df_severe_ds = metadata_df_severe_dataset.map(load_aug_dataset, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_severe_ds = metadata_df_severe_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_severe_ds = metadata_df_severe_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
metadata_df_severe_ds = metadata_df_severe_ds.prefetch(tf.data.AUTOTUNE)

metadata_df_severe_ds = metadata_df_severe_dataset.map(load_aug_dataset, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_severe_ds = metadata_df_severe_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_severe_ds = metadata_df_severe_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
metadata_df_severe_ds = metadata_df_severe_ds.prefetch(tf.data.AUTOTUNE)

metadata_df_moderate_ds = metadata_df_moderate_dataset.map(load_aug_dataset, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_moderate_ds = metadata_df_moderate_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
metadata_df_moderate_ds = metadata_df_moderate_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
metadata_df_moderate_ds = metadata_df_moderate_ds.prefetch(tf.data.AUTOTUNE)

severe_plus_moderate_augd = metadata_df_severe_ds.concatenate(metadata_df_moderate_ds)

combined_train_dataset = train_ds.concatenate(severe_plus_moderate_augd)

In [29]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

with tpu_strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")

    conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base.trainable = False

    max_pool_layer_0 = layers.MaxPooling2D(name="max_pool_0", pool_size=(2, 2), strides=(1, 1), padding="same")(conv_base.output)
    max_pool_layer_0 = layers.BatchNormalization()(max_pool_layer_0)

    conv2d_1 = layers.Conv2D(filters=100, kernel_size=4, strides=1, padding="same", activation="relu")(max_pool_layer_0)
    max_pool_layer_1 = layers.MaxPooling2D(name="max_pool_1", pool_size=(2, 2), strides=(1, 1), padding="same")(conv2d_1)
    max_pool_layer_1 = layers.BatchNormalization()(max_pool_layer_1)

    flattened_layer = layers.Flatten()(max_pool_layer_1)

    hidden_layer1 = layers.Dense(200, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
    hidden_layer1 = layers.BatchNormalization()(hidden_layer1)
    hidden_layer2 = layers.Dense(100, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer1)
    hidden_layer2 = layers.BatchNormalization()(hidden_layer2)
    hidden_layer3 = layers.Dense(50, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer2)
    hidden_layer3 = layers.BatchNormalization()(hidden_layer3)
    rsna_output = layers.Dense(3, activation="softmax")(hidden_layer3)
    model = tf.keras.Model(rsna_input, rsna_output)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_{0}_{1}.keras".format(condition_for_training, vertebrae_position))
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    """lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100, decay_rate=0.9)
    adamax_optimizer = tf.keras.optimizers.Adamax(learning_rate=lr_schedule)"""

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adamax", metrics=["accuracy"])
    # model.compile(loss="sparse_categorical_crossentropy", optimizer=adamax_optimizer, metrics=["accuracy"])

In [21]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

with tpu_strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")

    conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base.trainable = False
    
    conv2d_1 = layers.Conv2D(filters=64, kernel_size=7, strides=2, padding="same", activation="relu")(conv_base.output)
    max_pool_layer_1 = layers.MaxPooling2D(name="max_pool_1", pool_size=(3, 3), strides=(2, 2), padding="same")(conv2d_1)
    lrn_layer_1 = layers.Lambda(lambda X: tf.nn.local_response_normalization(X))(max_pool_layer_1)
    conv2d_2 = layers.Conv2D(filters=64, kernel_size=1, strides=1, padding="same", activation="relu")(lrn_layer_1)
    conv2d_3 = layers.Conv2D(filters=192, kernel_size=3, strides=1, padding="same", activation="relu")(conv2d_2)
    lrn_layer_2 = layers.Lambda(lambda X: tf.nn.local_response_normalization(X))(conv2d_3)
    max_pool_layer_2 = layers.MaxPooling2D(name="max_pool_2", pool_size=(3, 3), strides=(2, 2), padding="same")(lrn_layer_2)
    
    ## Inception layer 1
    conv2d_incpl_1_1 = layers.Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding="same", activation="relu")(max_pool_layer_2)
    conv2d_incpl_1_2 = layers.Conv2D(filters=128, kernel_size=(1, 1), strides=1, padding="same", activation="relu")(max_pool_layer_2)
    max_pool_layer_incpl_1 = layers.MaxPooling2D(name="max_pool_incpl_1", pool_size=(3, 3), strides=(1, 1), padding="same")(max_pool_layer_2)
    conv2d_incpl_1_3 = layers.Conv2D(filters=32, kernel_size=(1, 1), strides=1, padding="same", activation="relu")(max_pool_layer_2)
    conv2d_incpl_1_4 = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding="same", activation="relu")(conv2d_incpl_1_1)
    conv2d_incpl_1_5 = layers.Conv2D(filters=96, kernel_size=(5, 5), strides=1, padding="same", activation="relu")(conv2d_incpl_1_2)
    conv2d_incpl_1_6 = layers.Conv2D(filters=16, kernel_size=(1, 1), strides=1, padding="same", activation="relu")(max_pool_layer_incpl_1)
    concat_incpl_1 = layers.Concatenate(axis=3)([conv2d_incpl_1_3, conv2d_incpl_1_4, conv2d_incpl_1_5, conv2d_incpl_1_6])
    
    dropout_1 = layers.Dropout(0.4)
    
    flattened_layer = layers.Flatten()(concat_incpl_1)
    hidden_layer1 = layers.Dense(1000, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
    hidden_layer1 = layers.BatchNormalization()(hidden_layer1)
    
    rsna_output = layers.Dense(3, activation="softmax")(hidden_layer1)
    model = tf.keras.Model(rsna_input, rsna_output)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_{0}_{1}.keras".format(condition_for_training, vertebrae_position))
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    """lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100, decay_rate=0.9)
    adamax_optimizer = tf.keras.optimizers.Adamax(learning_rate=lr_schedule)"""

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adamax", metrics=["accuracy"])
    # model.compile(loss="sparse_categorical_crossentropy", optimizer=adamax_optimizer, metrics=["accuracy"])

In [22]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rsna_input          │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_10        │ (None, 128, 128,  │          0 │ rsna_input[0][0]  │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_5     │ (None, 128, 128,  │          7 │ rescaling_10[0][… │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_11        │ (None, 128, 128,  │          0 │ normalization_5[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 129, 129,  │          0 │ rescaling_11[0][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 64, 64,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 64, 64,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 64, 64,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 64, 64,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 64, 64,    │        512 │ block1a_se_excit

 Total params: 8,734,742 (33.32 MB)

 Trainable params: 4,683,171 (17.86 MB)

 Non-trainable params: 4,051,571 (15.46 MB)

In [ ]:
with tpu_strategy.scope():
    tf.config.run_functions_eagerly(False)
    history = model.fit(combined_train_dataset, class_weight={0:2, 1:1, 2:4}, epochs=20, validation_data=valid_ds
                        , callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20


2024-10-02 15:27:48.870612: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1727882869.727050     840 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(dff62cfd8afa0841:0:0), session_name()


  4/793 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.5859 - loss: 1.1456   

I0000 00:00:1727882886.554237     840 tpu_compile_op_common.cc:245] Compilation of dff62cfd8afa0841:0:0 with session name  took 16.827107044s and succeeded
I0000 00:00:1727882886.626396     840 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(dff62cfd8afa0841:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_3827490330085216969", property.function_library_fingerprint = 9587094457560686008, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1727882886.626445     840 tpu_compilation_cache_interface.cc:541] After adding entry for key dff62cfd8afa0841:0:

793/793 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8994 - loss: 0.6404

I0000 00:00:1727882929.172517     803 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(9b84aa8b2968c70a:0:0), session_name()
I0000 00:00:1727882933.085842     803 tpu_compile_op_common.cc:245] Compilation of 9b84aa8b2968c70a:0:0 with session name  took 3.913280911s and succeeded
I0000 00:00:1727882933.100126     803 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(9b84aa8b2968c70a:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_9231540245004798451", property.function_library_fingerprint = 7296738780756076653, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fin

793/793 ━━━━━━━━━━━━━━━━━━━━ 76s 65ms/step - accuracy: 0.8994 - loss: 0.6405 - val_accuracy: 0.0893 - val_loss: 1.7490
Epoch 2/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.6189 - loss: 0.9317 - val_accuracy: 0.0893 - val_loss: 1.8318
Epoch 3/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.6097 - loss: 0.9199 - val_accuracy: 0.0893 - val_loss: 1.9299
Epoch 4/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 56ms/step - accuracy: 0.6035 - loss: 0.9081 - val_accuracy: 0.0893 - val_loss: 1.9073
Epoch 5/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.6002 - loss: 0.8992 - val_accuracy: 0.0893 - val_loss: 1.4986
Epoch 6/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.6007 - loss: 0.8936 - val_accuracy: 0.0893 - val_loss: 1.4235
Epoch 7/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.6005 - loss: 0.8837 - val_accuracy: 0.0893 - val_loss: 2.3913
Epoch 8/20
793/793 ━━━━━━━━━━━━━━━━━━━━ 47s 58ms/step - accuracy: 0.5986 - loss: 0.8799 - val_accurac

In [21]:
with tpu_strategy.scope():
    model.evaluate(test_ds)

366/366 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.9556 - loss: 0.3082


In [22]:
with tpu_strategy.scope():
    model.save("keras_base_{0}_{1}.h5".format(condition_for_training, vertebrae_position))